<a href="https://colab.research.google.com/github/ollihansen90/zeugs/blob/main/Levenshtein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc -q https://raw.githubusercontent.com/ollihansen90/wortlisten/main/apfelsorten.txt
!wget -nc -q https://raw.githubusercontent.com/ollihansen90/wortlisten/main/laender.txt
!wget -nc -q https://raw.githubusercontent.com/ollihansen90/wortlisten/main/deutsche_woerter_kurz.txt
!wget -nc -q https://raw.githubusercontent.com/ollihansen90/wortlisten/main/staedtenamen.txt
!wget -nc -q https://raw.githubusercontent.com/ollihansen90/wortlisten/main/harrypotternames.txt

with open("apfelsorten.txt", "r") as file:
    apfelsorten = file.read().split("\n")

with open("laender.txt", "r") as file:
    laendernamen = file.read().split("\n")

with open("deutsche_woerter_kurz.txt", "r") as file:
    woerter = file.read().split("\n")

with open("staedtenamen.txt", "r") as file:
    staedte = file.read().split("\n")
with open("harrypotternames.txt", "r") as file:
    harrypotter = file.read().split("\n")

apfelsorten = list(set(apfelsorten))
laendernamen = list(set(laendernamen))
woerter = list(set(woerter))
staedte = list(set(staedte))
print(len(apfelsorten), "Apfelsorten geladen")
print(len(laendernamen), "Ländernamen geladen")
print(len(woerter), "Wörter geladen")
print(len(staedte), "Städte geladen")
print(len(harrypotter), "harrypotter geladen")

In [ ]:
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

def levenshtein(wort1, wort2, substitution=1, maxval=10000):
    wort1 = wort1.upper()
    wort2 = wort2.upper()
    data = np.zeros([len(wort2)+1, len(wort1)+1])
    data[0] = np.arange(0,len(wort1)+1)
    data[:,0] = np.arange(0,len(wort2)+1)

    for i in range(1, len(wort2)+1):
        for j in range(1, len(wort1)+1):
            if wort2[i-1]==wort1[j-1]:
                data[i,j] = data[i-1, j-1]
            else:
                data[i,j] = min([data[i-1,j]+1, data[i,j-1]+1, data[i-1,j-1]+substitution])
    return data

def draw_table():
    table = [list(" -"+wort1)]
    table.append(["-"]+ list(data[0]))
    for i in range(len(wort2)):
        table.append([wort2[i]]+ list(data[i+1]))

    plt.figure()
    plt.imshow(data)
    plt.axis("off")
    plt.show()

    print(tabulate(table, tablefmt="fancy_grid"))

wort1 = "Hund"
wort2 = "Wandel"
data = levenshtein(wort1, wort2, 1).astype(int)
draw_table()

In [ ]:
name = "GunMarit"
liste = []

for sorte in tqdm(apfelsorten):
    if 0.8*len(sorte)<len(sorte)<1.2*len(sorte):
        liste.append([sorte, levenshtein(name, sorte)[-1,-1]])

liste = sorted(liste, key=lambda x: x[1])
print(*liste[:20], sep="\n")

In [ ]:
def ngram(wort, n=3):
    output = []
    for i in range(len(wort)-n+1):
        output.append(wort[i:i+n])
    return output

def jaccard(wort1, wort2, n=2):
    ngram1 = set(ngram(wort1.upper(), n))
    ngram2 = set(ngram(wort2.upper(), n))

    schnitt = ngram1.intersection(ngram2)
    vereinigung = ngram1.union(ngram2)
    return len(schnitt)/len(vereinigung)

name = "Harry Potter"
n = 2 # if len(name)<5 else 3
liste = []
vorsortiert = []

for sorte in tqdm(harrypotter):
    jj = jaccard(name, sorte, n=n)
    vorsortiert.append([sorte, jj])

for sorte, jj in tqdm(sorted(vorsortiert, key=lambda x: x[1], reverse=True)[:20]):
    liste.append([sorte, jj, levenshtein(name, sorte)[-1,-1]])


liste = sorted(liste, key=lambda x: x[-1], reverse=False)
print(*liste, sep="\n")

In [ ]:
print(levenshtein("dustin", "stein"))
print(jaccard("dustin", "stein"))